# Image Classification using Convolutional Neural Network

In [1]:
# Importing necessary Libraries
import pandas as pd
import numpy  as np
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [2]:
# Rotating, Turning and Flipping our images 
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip=True)

In [3]:
# Loading and Pre-processing the dataset

test_datagen = ImageDataGenerator(rescale = 1./255)

# Creating the Training dataset
training_set = train_datagen.flow_from_directory("cat_and_dog_datasets/training_set/training_set",target_size = (64,64),batch_size = 32,class_mode = "binary")

# Creating the Testing dataset
testing_set = test_datagen.flow_from_directory("cat_and_dog_datasets/test_set/test_set",target_size = (64,64),batch_size = 32,class_mode = "binary")

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [4]:
# Building the Convolutinal Neural Network
classifier = Sequential()
classifier.add(Convolution2D(32,3,3,input_shape=(64,64,3),activation="relu"))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Flatten())
classifier.add(Dense(128,activation="relu"))
classifier.add(Dense(1,activation="sigmoid"))

In [5]:
# Compiling the CNN Model
classifier.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [6]:
# Training the CNN Model
classifier.fit(training_set,steps_per_epoch=10,epochs=25,validation_data=testing_set,validation_steps=2)

Epoch 1/25
10/10 [==============================] - 15s 1s/step - loss: 0.7321 - accuracy: 0.5125 - val_loss: 0.6867 - val_accuracy: 0.5312
Epoch 2/25
10/10 [==============================] - 12s 1s/step - loss: 0.6985 - accuracy: 0.4875 - val_loss: 0.6699 - val_accuracy: 0.5938
Epoch 3/25
10/10 [==============================] - 11s 1s/step - loss: 0.6859 - accuracy: 0.5156 - val_loss: 0.6897 - val_accuracy: 0.5312
Epoch 4/25
10/10 [==============================] - 10s 1s/step - loss: 0.6809 - accuracy: 0.5688 - val_loss: 0.6712 - val_accuracy: 0.5781
Epoch 5/25
10/10 [==============================] - 10s 991ms/step - loss: 0.6786 - accuracy: 0.5688 - val_loss: 0.6079 - val_accuracy: 0.7031
Epoch 6/25
10/10 [==============================] - 10s 1s/step - loss: 0.6777 - accuracy: 0.5500 - val_loss: 0.6662 - val_accuracy: 0.6406
Epoch 7/25
10/10 [==============================] - 10s 976ms/step - loss: 0.6631 - accuracy: 0.6187 - val_loss: 0.6701 - val_accuracy: 0.5938
Epoch 8/25
10/

In [13]:
# Classifying the New Images using the Trained CNN Model
import tensorflow as tf

def predict_image(imagepath):
    
    path    = imagepath
    predict = tf.keras.preprocessing.image.load_img(path, target_size=(64,64))
    predict_modified = tf.keras.preprocessing.image.img_to_array(predict)
    predict_modified = predict_modified / 255
    predict_modified = np.expand_dims(predict_modified,axis=0)
    
    result = classifier.predict(predict_modified)
    
    if result[0][0] >= 0.5:
        prediction = "dog"
        probability = result[0][0]
    else:
        prediction = "cat"
        probability = 1 - result[0][0]
    print(prediction)
    print("probability = "+str(probability))

predict_image("cat_and_dog_datasets/new_images/cat.5.jpg")
predict_image("cat_and_dog_datasets/new_images/dog.4001.jpg")        

1/1 [==============================] - 0s 21ms/step
cat
probability = 0.5015205442905426
1/1 [==============================] - 0s 21ms/step
dog
probability = 0.5751044
